In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [65]:
data = pd.read_csv('input/train.csv')
# data = pd.read_csv('input/train_min.csv')  # small data

X, y = data.iloc[:,2:].values, data.iloc[:,1].values

# std scaling
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [68]:
# cv: 0.8333937317304528(not scaled)
# logistic = SGDClassifier(loss='log', max_iter=50000, tol=1e-3, alpha=0.1)

#cv: 0.8492245702937613(not scaled)
# logistic = SGDClassifier(loss='log', max_iter=50000, tol=1e-3, alpha=0.01)

#cv: 0.8456784557444358(not scaled)
# logistic = SGDClassifier(loss='modified_huber', max_iter=5000, tol=1e-3, alpha=0.01)

#cv: 0.8494248938378591(not scaled)
# logistic = SGDClassifier(loss='log', max_iter=5000, tol=1e-3, alpha=0.01)

#cv: 0.8515980352095942(not scaled)
#    0.8595503242215752(scaled)
logistic = SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.01)

cls = logistic

# try grid-search
# params = { 'tol': [ 1e-8, 1e-9, 1e-10, 1e-15 ] }
#cls = GridSearchCV(estimator=logistic, param_grid=params, cv=5, scoring='roc_auc')

#print(cls.best_params_)
#print(cls.best_score_)

cv_scores = cross_val_score(cls, X_scaled, y, cv=5, scoring='roc_auc')

print(cv_scores)
print(np.average(cv_scores))

cls.fit(X_scaled, y)

[0.85575335 0.85629827 0.85840179 0.86425269 0.86304088]
0.8595493966729728


SGDClassifier(alpha=0.01, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=5000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=1e-07,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [69]:
# create final output
test = pd.read_csv('input/test.csv')
test_ids, test_x = test.iloc[:,0], test.iloc[:,1:]

test_x_scaled = scaler.transform(test_x)
test_y = cls.predict_proba(test_x_scaled)

In [70]:
# output the result
sub_ids = pd.DataFrame(test_ids)
sub_y = pd.DataFrame(pd.DataFrame(test_y).iloc[:,1])

sub = pd.DataFrame(np.hstack((sub_ids, sub_y)))
sub.to_csv('output.csv', header=['ID_code', 'target'], index=False)

output_check = pd.read_csv('output.csv')
output_check.head()

,ID_code,target
0,test_0,0.186199
1,test_1,0.233491
2,test_2,0.057496
3,test_3,0.196701
4,test_4,0.073267
